# HMM code golf

Hi! We've built an HMM for detecting [PCR chimeras](https://en.wikipedia.org/wiki/Chimera_(molecular_biology)). <br>
We want this HMM to run very fast. <br>
If you can speedup any of the two core HMM functions by >= 5x without changing the output, you get authorship on the paper. <br>

I want to focus on the core algorithms, so run with 1 thread. However, if you can improve the threading that's also a plus.

The main idea is that given a sequence query, we want to determine if it's more likely to observe the query from a database sequence plus a mutation rate, or by the combination of two database sequences plus mutation rate. The states are database sequences and the emission probabilities are affected by the mutation rates. The mutation rates can be either determined by an algorithm (baum welch approach) or given by the user (discrete bayesian approach). The core functions these two methods use are the same and the calls in the examples below use the baum welch approach.

I would like two specific functions optimized: get_chimera_probabilities and get_recombination_events. The core algorithms behind these functions are in https://github.com/MurrellGroup/CHMMera/blob/main/src/algorithms.jl . This is where the optimization should happen.

Thanks! <br>
\- Mark

In [ ]:
# install packages
using Pkg; Pkg.add("CSV"); Pkg.add("DataFrames"); Pkg.add("FASTX"); Pkg.add("BenchmarkTools"); Pkg.add("Revise")

In [ ]:
# grab HMM code
run(`git clone git@github.com:MurrellGroup/CHMMera.git`)

In [1]:
# import stuff
using CSV, DataFrames, FASTX, BenchmarkTools, Pkg
include("./test.jl")

Pkg.activate(".")
Pkg.develop(path="./CHMMera/")
using Revise
using CHMMera

  Activating project at `~/ben/speedy_hmm`
   Resolving package versions...
  No Changes to `~/ben/speedy_hmm/Project.toml`
  No Changes to `~/ben/speedy_hmm/Manifest.toml`


In [2]:
# when optimizing core algos, keep this at 1
Threads.nthreads()

1

## Run this cell to prepare the input data

In [3]:
function read_fasta(filepath::String)
    reader = FASTX.FASTA.Reader(open(filepath, "r"))
    fasta_in = [record for record in reader]
    close(reader)
    return [String(FASTX.FASTA.description(rec)) for rec in fasta_in],
    [uppercase(String(FASTX.FASTA.sequence(rec))) for rec in fasta_in]
end

#These functions take an IgBlast alignment, and thread it onto the reference alignment, transposing gaps
#This currently discards insertions relative to the sequence that IgBlast says its closest to.  
#If other references sequences had similar insertions, then this is discarded signal
#I don't think this is going to make much practical difference though
function thread_seq(v_sequence_alignment::AbstractString, 
                    v_germline_alignment::AbstractString, 
                    v_call::AbstractString, 
                    refseqs::Vector, 
                    degapped_refs::Vector, 
                    refname2ind::Dict, 
                    ali_length::Int):: String

    dg = degap(v_germline_alignment)
    gapped_full_ref = refseqs[refname2ind[v_call]]
    # Because igblast gives a local alignment, sometimes missing ends:
    matchrange = findfirst(dg, degapped_refs[refname2ind[v_call]])
    gapped_full_ref = refseqs[refname2ind[v_call]]

    keep_pos = collect(v_germline_alignment) .!= '-';
    no_inserts = collect(v_sequence_alignment)[keep_pos];
    no_inserts = vcat(['N' for i in 1:matchrange[1] - 1], no_inserts)#
    threaded = fill('-',ali_length);
    pos = 1
     
    for i in 1:ali_length
        if pos <= length(no_inserts)
            if gapped_full_ref[i] != '-'
                threaded[i] = no_inserts[pos]
                pos += 1
            end
        else
            threaded[i] = 'N'
        end
    end
    return join(threaded)
end

function thread_all(queries::Vector{String},q2refs::Vector{String},q2ref_names::Vector{String},refseqs::Vector{String},degapped_refs::Vector{String},refname2ind::Dict{String, Int64},ali_length::Int64)::Vector{String}
    threaded = Vector{String}(undef, length(queries))
    Threads.@threads for i in eachindex(queries)
        threaded[i] = thread_seq(queries[i], q2refs[i], q2ref_names[i], refseqs, degapped_refs, refname2ind, ali_length)
    end
    return threaded
end

function degap(s::String)
    return replace(s, "-" => "")
end

assignments = CSV.read("10k_IGH_sim.tsv", DataFrame, delim = "\t");
refnames, refseqs = read_fasta("IGHV_sim.fasta");
refseqs = uppercase.(refseqs);
# setting up the inputs for threading
queries = uppercase.(Array(assignments[!,"v_sequence_alignment"]))
q2refs = uppercase.(Array(assignments[!,"v_germline_alignment"]))
q2ref_names = map(x->String(split(x, ",")[1]), Array(assignments[!,"v_call"]))
degapped_refs = degap.(refseqs);
refname2ind = Dict(zip(refnames,collect(eachindex(refnames))))
ali_length = maximum(length.(refseqs))
@assert length.(q2refs) == length.(queries)
threaded = thread_all(queries,q2refs,q2ref_names,refseqs,degapped_refs,refname2ind,ali_length);

# Target 1: get_chimera_probabilities

This is the core functionality of the tool. <br>
It runs parameter estimation, then the forward algorithm to get the probability of chimerism

In [4]:
@benchmark CHMMera.get_chimera_probabilities(threaded, refseqs)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  3.970 s …  3.981 s  ┊ GC (min … max): 1.77% … 1.70%
 Time  (median):     3.975 s             ┊ GC (median):    1.73%
 Time  (mean ± σ):   3.975 s ± 7.766 ms  ┊ GC (mean ± σ):  1.73% ± 0.05%

  █                                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.97 s        Histogram: frequency by time        3.98 s <

 Memory estimate: 9.08 GiB, allocs estimate: 2770181.

# Target 2: get_recombination_events
This function runs the viterbi algorithm to get additional information about the chimeras <br>
It's slower than the first target

In [5]:
@benchmark CHMMera.get_recombination_events(threaded, refseqs, true, [0.0], 0.05, 1/300, true, true)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 9.130 s (2.46% GC) to evaluate,
 with a memory estimate of 22.59 GiB, over 13917331 allocations.

# Check outputs
After you modify the functions, make sure you didn't change the outputs. <br>
Run the two test functions below and make sure you get a PASS 

In [6]:
chimera_probs = CHMMera.get_chimera_probabilities(threaded, refseqs)
test_chimera_probs(chimera_probs)
chimera_recombs = CHMMera.get_recombination_events(threaded, refseqs, true, [0.0], 0.05, 1/300, true, true)
test_chimera_recombs(chimera_recombs)

┌ Info: PASS CHIMERA PROBS 👍
└ @ Main /home/mchernys/ben/speedy_hmm/test.jl:5
┌ Info: PASS CHIMERA RECOMBS 👍
└ @ Main /home/mchernys/ben/speedy_hmm/test.jl:18
